In [1]:
# import necessary libraries

import pandas as pd
from pandas import read_csv
from pandas.core.common import SettingWithCopyWarning
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv(r'C:\Users\valer\Desktop\Projects\honolu\for_pickle.csv')

In [3]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
df.head()

,rooms*,n_area_log,n_price_log
0,20.0,7.090077,7.824046
1,1.0,6.682109,7.598399
2,2.0,6.210600,7.492760
3,10.0,6.785588,7.781139
4,9.0,7.350516,8.715357


In [5]:
# preparing data for the train_test_split
X = df.iloc[:,:-1]
Y = df.iloc[:, -1]

# splitting data into test and train splits
from sklearn.model_selection import train_test_split

# split data into test (20%) and train (80%) parts
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
print("x train: ",X_train.shape)
print("x test: ",X_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

x train:  (2496, 2)
x test:  (624, 2)
y train:  (2496,)
y test:  (624,)


In [6]:
# features scaling
sc = StandardScaler()
X_train_st = sc.fit_transform(X_train)
X_test_st = sc.transform(X_test)

print('X_train: ', X_train_st)
print()
print('X_test: ', X_test_st)

X_train:  [[-0.36861045 -1.54939008]
 [-0.36861045 -1.09673648]
 [-0.36861045 -1.54939008]
 ...
 [-0.36861045 -0.83779942]
 [-0.36861045 -0.83292996]
 [-0.64210977 -2.09896494]]

X_test:  [[ 0.17838817  1.00245042]
 [ 0.17838817  1.00245042]
 [-0.64210977 -0.6290425 ]
 ...
 [ 0.17838817  0.18314995]
 [-0.36861045 -1.54939008]
 [-0.36861045 -1.12445779]]


In [11]:
X_test_st

array([[ 0.17838817,  1.00245042],
       [ 0.17838817,  1.00245042],
       [-0.64210977, -0.6290425 ],
       ...,
       [ 0.17838817,  0.18314995],
       [-0.36861045, -1.54939008],
       [-0.36861045, -1.12445779]])

In [7]:
# for Random Forest with the best hyperparameters
rf = RandomForestRegressor(n_estimators= 600,min_samples_split= 2,min_samples_leaf= 4,
                           max_features= 'auto',max_depth= 40, bootstrap= True, random_state = 42)

# fir train data to the model
rf.fit(X_train_st, y_train)

# find predictions
predictions = rf.predict(X_test_st)

In [18]:
rf.predict([[2,200]])

array([8.19854083])

In [8]:
from sklearn.model_selection import cross_val_score

#calculate cross validated R2
R2 = cross_val_score(rf, X=X_train_st, y=y_train,cv=10, scoring='r2').mean()  

#display adjusted R-squared
#calculate Adj R2
n= len(X_train)
p = len(X_train.columns)
adj_R2 = 1- ((1-R2) * (n-1)/(n-p-1))

In [9]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("R2 score =", round(metrics.r2_score(y_test, predictions), 2))
print("adjR2 score =", round(adj_R2, 2))

MAE: 0.10876193968431636
MSE: 0.059839321950595326
RMSE: 0.24462077170713717
R2 score = 0.65
adjR2 score = 0.73


In [10]:
import pickle
with open('Forest_model.pkl', 'wb') as file:
    pickle.dump(rf, file)